# ဆင်ဆာမတစ်ကွက်ကိရိယာ အသုံးပြုမှု ဥပမာ


In [1]:
import json
import os

from dotenv import load_dotenv

from IPython.display import display, HTML

from typing import Annotated
from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## ပလပ်ဂင်များ ဖန်တီးခြင်း    
Semantic Kernel သည် ပလပ်ဂင်များကို အေးဂျင့်က ခေါ်သုံးနိုင်သော ကိရိယာများအဖြစ် အသုံးပြုသည်။ ပလပ်ဂင်တစ်ခုတွင် `kernel_functions` များစွာကို အုပ်စုအဖြစ် ပါဝင်နိုင်သည်။

အောက်ပါ ဥပမာတွင်၊ `DestinationsPlugin` ကို ဖန်တီးထားပြီး၊ အောက်ပါ လုပ်ဆောင်ချက်နှစ်ခု ပါဝင်သည် -  
1. `get_destinations` function ကို အသုံးပြုပြီး ခရီးစဉ်များ၏ စာရင်းကို ပေးသည်  
2. `get_availabilty` function ကို အသုံးပြုပြီး ခရီးစဉ်တစ်ခုစီအတွက် ရရှိနိုင်မှု စာရင်းကို ပေးသည်  


In [2]:
# Define a sample plugin for the sample
class DestinationsPlugin:
    """A List of Destinations for vacation."""

    @kernel_function(description="Provides a list of vacation destinations.")
    def get_destinations(self) -> Annotated[str, "Returns the vacation destinations."]:
        return """
        Barcelona, Spain
        Paris, France
        Berlin, Germany
        Tokyo, Japan
        New York, USA
        """

    @kernel_function(description="Provides the availability of a destination.")
    def get_availability(
        self, destination: Annotated[str, "The destination to check availability for."]
    ) -> Annotated[str, "Returns the availability of the destination."]:
        return """
        Barcelona - Unavailable
        Paris - Available
        Berlin - Available
        Tokyo - Unavailable
        New York - Available
        """

## Client ကို ဖန်တီးခြင်း

ဤနမူနာတွင် [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ကို အသုံးပြု၍ LLM ကို အသုံးပြုမည်ဖြစ်သည်။

`ai_model_id` ကို `gpt-4o-mini` အဖြစ် သတ်မှတ်ထားသည်။ GitHub Models marketplace တွင် ရရှိနိုင်သည့် အခြားမော်ဒယ်တစ်ခုကို ပြောင်းလဲကြည့်ပြီး မတူညီသော ရလဒ်များကို ကြည့်ရှုနိုင်ပါသည်။

GitHub Models အတွက် `base_url` ကို အသုံးပြုရန်လိုအပ်သည့် `Azure Inference SDK` ကို အသုံးပြုရန်အတွက်၊ Semantic Kernel အတွင်းရှိ `OpenAIChatCompletion` connector ကို အသုံးပြုမည်ဖြစ်သည်။ အခြားမော်ဒယ်ပံ့ပိုးသူများအတွက် Semantic Kernel ကို အသုံးပြုနိုင်ရန် [ရရှိနိုင်သော အခြား connector များ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) လည်း ရှိပါသည်။


In [3]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.getenv("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## အေးဂျင့်ကို ဖန်တီးခြင်း  
အခုတော့ အေးဂျင့်နာမည်နှင့် လမ်းညွှန်ချက်များကို သတ်မှတ်ပြီး အေးဂျင့်ကို ဖန်တီးပါမည်။

သင်သည် အေးဂျင့်၏ တုံ့ပြန်မှုအမျိုးမျိုးကို ကြည့်ရှုနိုင်ရန် ဒီဆက်တင်များကို ပြောင်းလဲနိုင်ပါသည်။


In [4]:
# Create the agent
agent = ChatCompletionAgent(
    service=chat_completion_service,
    name="TravelAgent",
    instructions="Answer questions about the travel destinations and their availability.",
    plugins=[DestinationsPlugin()],
)

## အေးဂျင့်ကို အလုပ်လုပ်စေခြင်း

အခုတော့ AI အေးဂျင့်ကို အလုပ်လုပ်စေမယ်။ ဒီနမူနာမှာ `user_input` ထဲမှာ မက်ဆေ့ဒ်နှစ်ခုထည့်ပြီး အေးဂျင့်က ဆက်လက်မေးမြန်းမှုတွေကို ဘယ်လိုတုံ့ပြန်မလဲဆိုတာ ပြသနိုင်ပါတယ်။

အေးဂျင့်က ရရှိနိုင်တဲ့နေရာများစာရင်းကို ရယူဖို့ မှန်ကန်တဲ့ function ကို ခေါ်ပြီး တစ်နေရာရာရရှိနိုင်မှုကို အတည်ပြုသင့်ပါတယ်။

သင့်အနေနဲ့ `user_inputs` ကို ပြောင်းလဲပြီး အေးဂျင့်က ဘယ်လိုတုံ့ပြန်မလဲဆိုတာ ကြည့်ရှုနိုင်ပါတယ်။


In [6]:
user_inputs = [
    "What destinations are available?",
    "Is Barcelona available?",
    "Are there any vacation destinations available not in Europe?",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()

ServiceResponseException: ("<class 'semantic_kernel.connectors.ai.open_ai.services.open_ai_chat_completion.OpenAIChatCompletion'> service failed to complete the prompt", AuthenticationError("Error code: 401 - {'error': {'code': 'unauthorized', 'message': 'Bad credentials', 'details': 'Bad credentials'}}"))


---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါရှိနိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာရှိသော ရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအမှားများ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
